# Monte Carlo Simulation for LCOE Analysis with Hierarchical Aggregation

## Introduction
This notebook performs a Monte Carlo simulation to analyze the Levelized Cost of Energy (LCOE) by simulating detailed cost elements and aggregating them hierarchically.

## Import Libraries
Importing necessary libraries for data handling, numerical computations, and visualization.

In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import logging

#Number of years is equal to Years of Mission + 1 for pre-launch costs 
NUM_OF_TOTAL_YEARS = 21
NUM_OF_ITERATIONS = 10


# Trying to do some good kind of programing from here with objects

## BaseComponent
These are classes for the lowest level and include information that is to the left of the excel





In [110]:
class BaseComponent:
    def __init__(self, name, parents, unit, distribution, time_for_determination, low, high, sd, mean):
        """
        Initialize a BaseComponent object with specified attributes.

        Args:
        - name (str): Name of the component.
        - parents (str): Parents of the component.
        - unit (str): Unit of measurement for the component.
        - distribution (str): Distribution type for cost calculation.
        - time_for_determination (str or int): Time specification for cost determination.

        Attributes:
        - name (str): Name of the component.
        - parents (str): Parents of the component.
        - unit (str): Unit of measurement for the component.
        - distribution (str): Distribution type for cost calculation.
        - time_for_determination (str or int): Time specification for cost determination.
        - cost (np.ndarray): Array of costs calculated based on time_for_determination.
        """
        self.name = name
        self.parents = parents
        self.unit = unit
        self.distribution = distribution
        self.time_for_determination = time_for_determination
        self.low = low
        self.high = high
        self.sd = sd
        self.mean = mean
        self.cost = self._get_cost_array()

    def __repr__(self):
        """
        Return a string representation of the BaseComponent object.
        """
        return f"BaseComponent(name='{self.name}', unit='{self.unit}', distribution='{self.distribution}', parents='{self.parents}')"

    def __add__(self, other):
        """
        Define addition behavior for BaseComponent objects.
        
        Args:
        - other (BaseComponent): Another BaseComponent object to add.

        Returns:
        - list: Combined cost array element-wise.

        Raises:
        - TypeError: If 'other' is not an instance of BaseComponent.
        """
        if isinstance(other, BaseComponent):
            combined_cost = np.array([[self.cost[j, i] + other.cost[j, i] for i in range(self.cost.shape[1])] for j in range(self.cost.shape[0])])
            return combined_cost
        else:
            raise TypeError(f"Unsupported operand type(s) for +: '{type(self)}' and '{type(other)}'")

    def __mul__(self, other):
        """
        Define multiplication behavior for BaseComponent objects.
        
        Args:
        - other (BaseComponent): Another BaseComponent object to multiply.

        Returns:
        - list: Combined cost array element-wise.

        Raises:
        - TypeError: If 'other' is not an instance of BaseComponent.
        """
        if isinstance(other, BaseComponent):
            combined_cost = np.array([[self.cost[j, i] * other.cost[j, i] for i in range(self.cost.shape[1])] for j in range(self.cost.shape[0])])
            return combined_cost
        else:
            raise TypeError(f"Unsupported operand type(s) for *: '{type(self)}' and '{type(other)}'")

    def _get_cost_array(self):
        """
        Generate an array of costs based on time_for_determination and distribution.
        
        Returns:
        - np.ndarray: Array of costs calculated based on time_for_determination.
        """
        cost = np.zeros((NUM_OF_ITERATIONS, NUM_OF_TOTAL_YEARS))
        try:
            for j in range(cost.shape[0]):
                if self.time_for_determination in [0, 'Y0', 'YO', 'y0']:
                    cost[j, 0] = generate_random_value(self.distribution, mean=self.mean, sd=self.sd, low=self.low, high=self.high)
                elif 'Yearly' in self.time_for_determination:
                    for i in range(1, cost.shape[1]):
                        cost[j, i] = generate_random_value(self.distribution, mean=self.mean, sd=self.sd, low=self.low, high=self.high)
                else:
                    raise ValueError(f"Unrecognized time_for_determination value: '{self.time_for_determination}'")
        except ValueError as ve:
            logging.error(f"ValueError occurred: {ve}")
            logging.info('Normal distribution is assumed.')
            cost[:] = generate_random_value('Normal')
        except TypeError as te:
            logging.error(f"TypeError occurred: {te}")
            logging.info('Normal distribution is assumed.')
            cost[:] = generate_random_value('Normal')
        except Exception as e:
            logging.error(f"Error while calculating cost for component '{self.name}': {e}")
            logging.info('Normal distribution is assumed.')
            cost[:] = generate_random_value('Normal')
        return cost


## CollectorComponent
These include classes that contain other BaseComponets or collector components

In [111]:
class CollectorClass:
    """
    Represents a collector class for managing parts and their costs.
    """

    def __init__(self, name):
        """
        Initialize a CollectorClass instance.

        Args:
            name (str): The name of the collector class.
        """
        self.name = name
        self.parts = {}
        self.cost = None

    def add_part(self, parts, operation="+"):
        """
        Add parts to the collector class with specified operation on costs.
        
        Args:
            parts (list or BaseComponent or CollectorClass): Parts to add.
            operation (str, optional): Operation to perform on costs ('+', '*', '-'). Defaults to '+'.
        Raises:
            TypeError: If parts are not instances of BaseComponent or CollectorClass.
            ValueError: If parts have different length cost arrays.

        """
        if isinstance(parts, BaseComponent) or isinstance(parts, CollectorClass):
            parts = [parts]

        for part in parts:
            if isinstance(part, BaseComponent) or isinstance(part, CollectorClass):
                self.parts[part.name] = part
            else:
                raise TypeError("Only instances of BaseComponent or CollectorClass can be added as parts.")

        # Initialize cost if it's None
        if self.cost is None:
            self.cost = parts[0].cost[:, :]  # Copy the cost array from the first part
        elif len(self.cost) != len(parts[0].cost):
            raise ValueError("The length of the cost arrays are not the same. Cannot add parts with different cost array lengths.")

        # Perform specified operation on costs element-wise
        elif operation == "+":
            for part in parts:
                self.cost = np.array([[self.cost[j, i] + part.cost[j, i] for i in range(self.cost.shape[1])] for j in range(self.cost.shape[0])])

        elif operation == "*":
            for part in parts:
                self.cost = np.array([[self.cost[j, i] * part.cost[j, i] for i in range(self.cost.shape[1])] for j in range(self.cost.shape[0])])

        elif operation == "-":
            for part in parts:
                self.cost = np.array([[self.cost[j, i] - part.cost[j, i] for i in range(self.cost.shape[1])] for j in range(self.cost.shape[0])])


    def set_cost(self, cost):
        """
        Manually set the cost of the collector class.

        Args:
            cost (list): The cost array to set.
        """
        self.cost = cost
        print("Cost is manually changed")

    def __repr__(self):
        """
        Return a string representation of the CollectorClass instance.
        """
        return f"CollectorClass(name='{self.name}', parts={list(self.parts.keys())})"


## Additional helper functions

In [112]:
def extract_base_components(excel_path: str, sheet_name: str):
    """
    Extract base components from an Excel file.

    Args:
        excel_path (str): Path to the Excel file.
        sheet_name (str): Name of the sheet to read.

    Returns:
        dict: Dictionary of base components.
    """

    data = pd.read_excel(excel_path, sheet_name, skiprows=2)  # Skip the first two rows

    # Filter relevant columns and rows
    data = data.rename(columns={
        'Unnamed: 0': 'Include',
        'Unnamed: 1': 'Primary',
        'Unnamed: 2': 'Secondary',
        'Unnamed: 3': 'Tertiary',
        'Unnamed: 4': 'Quaternary',
        'Unnamed: 5': 'Quintenary',
        'Unnamed: 6': 'Units',
        'Unnamed: 7': 'Distribution',
        'Unnamed: 8': 'Time for determination (Year)',
        'Unnamed: 9': 'Lower Limit',  # Assuming 'Unnamed: 9' is the current name
        'Unnamed: 10': 'Upper Limit',  # Assuming 'Unnamed: 10' is the current name
        'Unnamed: 11': 'SD',  # Assuming 'Unnamed: 11' is the current name
        'Unnamed: 12': 'Mean',  # Assuming 'Unnamed: 12' is the current name
        'Unnamed: 13': 'Most Likely',  # Assuming 'Unnamed: 13' is the current name
        'Unnamed: 14': 'Comments'
    })
    try:
        # Read the Excel file
        df = data
        
        # Check if 'Include' column exists
        if 'Include' not in df.columns:
            raise KeyError("'Include' column not found in the Excel file.")
        
        # Filter the relevant rows and columns
        filtered_df = df[df['Include'] == 'Include']
        
        # Create a dictionary to store the base components
        base_components = {}
        
        for _, row in filtered_df.iterrows():
            # Concatenate the primary to quintenary fields to form the component name
            name = ' > '.join(filter(pd.notna, [row['Primary'], row['Secondary'], row['Tertiary'], row['Quaternary'], row['Quintenary']]))
            parents = ' > '.join(name.split(' > ')[:-1])
            name = name.split(' > ')[-1]
            unit = row['Units']
            distribution = row['Distribution']
            time_for_determination = row['Time for determination (Year)']
            low= row['Lower Limit']
            high = row['Upper Limit']
            sd = row['SD']
            mean = row['Mean']
            # most_likely = row['Most Likely']
            # comments = row['Comments']

            
            base_components[name] = BaseComponent(name, parents, unit, distribution, time_for_determination, low, high, sd, mean)
        
        return base_components
    
    except Exception as e:
        print(f"Error occurred: {e}")
        return []

def generate_random_value(distribution, mean=0, sd=1, low=0, high=1, shape=1, scale=1, count=1):
    """
    Generate a random value based on the specified distribution.
    
    Parameters:
        distribution (str): The name of the distribution (e.g., 'Uniform', 'Normal', 'Exponential').
        mean (float): The mean value (used for 'Normal', 'Gamma', 'Log-normal' distributions).
        sd (float): The standard deviation (used for 'Normal', 'Log-normal' distributions).
        low (float): The lower bound (used for 'Uniform' distribution).
        high (float): The upper bound (used for 'Uniform' distribution).
        shape (float): The shape parameter (used for 'Gamma', 'Weibull' distributions).
        scale (float): The scale parameter (used for 'Gamma', 'Exponential', 'Weibull' distributions).
        count (int): The number of occurrences (used for 'Poisson' distribution).
    
    Returns:
        float: A random value based on the specified distribution.
    """
    if distribution == 'Uniform':
        return np.random.uniform(low, high)
    elif distribution == 'Normal' or distribution == 'nORMAL':  # Handling case sensitivity
        return np.random.normal(mean, sd)
    elif distribution == 'Exponential':
        return np.random.exponential(scale)
    elif distribution == 'Poisson':
        return np.random.poisson(count)
    elif distribution == 'Gamma':
        return np.random.gamma(shape, scale)
    elif distribution == 'Beta':
        return np.random.beta(shape, scale)
    elif distribution == 'Weibull':
        return np.random.weibull(shape) * scale
    elif distribution == 'Log-normal':
        return np.random.lognormal(mean, sd)
    elif distribution == 'Linear':  # Assuming Linear as a uniform distribution
        return np.random.uniform(low, high)
    elif distribution == 'N/A': # this will mean it will be a umber
        return 20  # Assuming 'N/A' as 0 contribution to cost
    else:
        raise ValueError(f"Unsupported distribution: {distribution}")

## Using the classes and the functions
- If a new row is added to the excel for costs, this section will have to be updated

In [113]:
# Example usage
excel_path = 'data/LCOEBENCE(1).xlsx'
base_components = extract_base_components(excel_path, "Sheet2")
for key, value in base_components.items():
    print(value) # you can double check here that all the info that you need is there

repairs = CollectorClass("Repairs")
repairs.add_part(base_components["Material Repairs"])


launch = CollectorClass("Launch")
parts = [base_components["Repair Number of Launches"], base_components["Repair Launching cost"]]
launch.add_part(parts, "*")

maintenance = CollectorClass("Maintenance")
parts = [launch, repairs]
maintenance.add_part(parts)

fuel = CollectorClass("Fuel")
parts = [base_components["Fuel use"], base_components["Fuel cost"]]
fuel.add_part(parts, "*")

energy = CollectorClass("Energy")
parts = [base_components["Installed Capacity"], base_components["Efficiency (n)"], base_components["#days in year x"], base_components["Hours in a day"]]
energy.add_part(parts, "*")
energy.add_part(base_components["Energy consumption"], "-")

transport = CollectorClass("Transport")
parts = [base_components["Distance of transport"], base_components["Cost of Fuel"]]
transport.add_part(parts, "*")

manufacture_panels = CollectorClass("Manufacture Panels")
parts = [base_components['Number of panels'], base_components['Raw Materials'], base_components['Processing']]
manufacture_panels.add_part(parts, "*")
manufacture_panels.add_part(transport)

deployment = CollectorClass("Deployment")
deployment.add_part(base_components['Logistics Costs'])


launch = CollectorClass("Launch")
parts = [base_components['Number of Launches'], base_components['Launching cost']]


print(manufacture_panels)

print(base_components["Repair Number of Launches"]+ base_components["Repair Launching cost"])




ERROR:root:ValueError occurred: Unsupported distribution: nan
ERROR:root:ValueError occurred: Unsupported distribution: nan
ERROR:root:ValueError occurred: Unsupported distribution: nan
ERROR:root:ValueError occurred: Unsupported distribution: nan
ERROR:root:Error while calculating cost for component 'Depreciation of Assets': Range exceeds valid bounds
ERROR:root:ValueError occurred: Unsupported distribution: nan
ERROR:root:ValueError occurred: Unsupported distribution: nan


BaseComponent(name='Material Repairs', unit='£', distribution='Exponential', parents='Asset > Maintenance > Repairs')
BaseComponent(name='Repair Number of Launches', unit='Count', distribution='Poisson', parents='Asset > Maintenance > Repairs > Launch')
BaseComponent(name='Repair Launching cost', unit='£', distribution='Gamma', parents='Asset > Maintenance > Repairs > Launch')
BaseComponent(name='Fuel use', unit='m3', distribution='Normal', parents='Asset > Fuel')
BaseComponent(name='Fuel cost', unit='£/m3', distribution='Normal', parents='Asset > Fuel')
BaseComponent(name='Total emissions', unit='CO2e/MWh', distribution='Normal', parents='Asset')
BaseComponent(name='Emission Cost', unit='£/CO2e', distribution='Normal', parents='Asset')
BaseComponent(name='Installed Capacity', unit='MW', distribution='nan', parents='Asset > Energy > Energy Genrated')
BaseComponent(name='Efficiency (n)', unit='%', distribution='nan', parents='Asset > Energy > Energy Genrated')
BaseComponent(name='#days 